<img align="left" src = https://noirlab.edu/public/media/archives/logos/svg/logo250.svg width=250 style="background-color:white; padding:10px" alt="Rubin Observatory logo, a graphical representation of turning stars into data.">
<br>
Contact author(s): Christina Williams, Douglas Tucker <br>
Last verified to run: 2025-07-15 <br>
LSST Science Pipelines version: r29.1.1 <br>
Container Size: medium <br>
Targeted learning level: beginner <br>

**Description:** Explore the LSST Pipeline data products relevant to galaxy science. Explore the available measurements of galaxy photometry produced by the LSST pipelines and their applications.

**Skills:** Basic understanding of photometric measurement methods typically used to characterize galaxies.

**LSST Data Products:** deepCoadd, objectTable

**Packages:** `lsst.afw`, `lsst.daf`, `lsst.rsp`, `lsst.geom`, `lsst.sphgeom`, `lsst.gauss2d`, `astropy`, `photutils`, `pyvo`, `galsim`

**Credit:**
This notebook includes selections of DP0.2 tutorial notebook 03a (image display and manipulation) and DP1 tutorial notebook 303.1 (galaxy photometry). 

**Get Support:**
Find DP1-related documentation and resources at <a href="https://dp1.lsst.io">dp1.lsst.io</a>.
Questions are welcome as new topics in the 
<a href="https://community.lsst.org/c/support/dp1">Support - Data Preview 1 Category</a> 
of the Rubin Community Forum. 
Rubin staff will respond to all questions posted there.

# 1. Introduction

This tutorial will overview some data products produced by the LSST pipeline with applications for galaxy science. Section 2 will overview the data products (deepCoadd images, how to access them, and some simple image display and manipulation) and the catalog of objects that made from detections found in those coadds (the objectTable). Section 3 will then overview some of the photometry and shape measurements that are produced of galaxies in deepCoadds.


## 1.1 Import packages


`numpy` is a fundamental package for scientific computing with arrays in Python
(<a href="https://numpy.org">numpy.org</a>).

`matplotlib` is a comprehensive library for creating static, animated, and
interactive visualizations in Python 
(<a href="https://matplotlib.org/">matplotlib.org</a>; 
<a href="https://matplotlib.org/stable/gallery/index.html">matplotlib gallery</a>).

From the `lsst` package, modules for accessing the TAP service, and image display functions from `afw` are imported (<a href="https://pipelines.lsst.io/">pipelines.lsst.io</a>). Also import some geometric functions to help plot photometric apertures.

From the `pyvo` package, import some functions that will enable using the image cutout tool.

Finally, from `astropy` and `photutils` import packages to enable plotting images and drawing shapes on images with WCS information. Also import `galsim` which is the galaxy simulation package, which was used to build the DP0.2 simulated images, and is useful for reconstructing the sersic profiles that are used to model the galaxies in Rubin images.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.afw.display as afwDisplay
from lsst.afw.image import ExposureF
import lsst.afw.geom.ellipses as ellipses
from lsst.afw.fits import MemFileManager

from lsst.rsp import get_tap_service
from lsst.rsp.utils import get_pyvo_auth
from lsst.rsp.service import get_siav2_service
from lsst.daf.butler import Butler

import lsst.geom as geom
import lsst.sphgeom as sphgeom
from lsst.gauss2d import Ellipse, EllipseMajor

from pyvo.dal.adhoc import DatalinkResults, SodaQuery

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from photutils.aperture import SkyCircularAperture, SkyEllipticalAperture

import galsim as gs

## 1.2 Define functions and parameters


### 1.2.1 Define plot defaults

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

plt.style.use('tableau-colorblind10')


### 1.2.2 Instantiate the Butler

Instantiate the Butler, and assert that it exists.

In [ ]:
butler = Butler('dp1', collections="LSSTComCam/DP1")
assert butler is not None

### 1.2.3 Initialize the TAP service
Get an instance of the TAP service, and assert that it exists.

In [ ]:
service = get_tap_service("tap")
assert service is not None

### 1.2.4 Image cutout function
Define a function to generate an image cutout, using the Rubin image cutout service. Further information about the cutout tool can be found in DP1 tutorial notebook 103.4 that demonstrates the Rubin image cutout service. 

In [ ]:
def make_image_cutout(ra, dec, cutout_size=0.01):
    """
    Wrapper function to generate a cutout using the cutout tool

    Parameters
    ----------
    tap_service : an instance of the TAP service
    ra, dec : 'float'
        the ra and dec of the cutout center
    cutout_size : 'float', optional
        edge length in degrees of the cutout

    Returns
    -------
    mem : 'object'
        An lsst.afw.fits._fits.MemFileManager object containing the
        cutout image bytes returned by the cutout tool. The
        cutout image bytes can be accessed as an LSST awf image
        exposure class format such as: cutout = ExposureF(mem)
    """
    service = get_siav2_service("dp1")
    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)

    circle = (ra, dec, cutout_size)
    results = service.search(pos=circle, calib_level=3)
    table = results.to_table()
    tx = np.where((table['dataproduct_subtype'] == 'lsst.deep_coadd')
                  & (table['lsst_band'] == 'r'))[0]

    datalink_url = results[tx[0].item()].access_url
    dl = DatalinkResults.from_result_url(datalink_url, session=get_pyvo_auth())

    sq = SodaQuery.from_resource(dl,
                                 dl.get_adhocservice_by_id("cutout-sync"),
                                 session=get_pyvo_auth())

    sq.circle = (spherePoint.getRa().asDegrees() * u.deg,
                 spherePoint.getDec().asDegrees() * u.deg,
                 cutout_size * u.deg)
    cutout_bytes = sq.execute_stream().read()
    mem = MemFileManager(len(cutout_bytes))
    mem.setData(cutout_bytes, len(cutout_bytes))

    return mem

# 2. Data Products relevant for galaxies

## 2.1  `deepCoadd` images

This section will examine coadded images made up of multiple exposures that are combined over the course of the LSST survey, and demonstrate some simple display and manipulation. 

Start by grabbing a `deepCoadd` image, which is a full 4k x 4k pixel coadd "patch". Retrieve the image using the butler, using an RA and dec. Choose coordinates in the Extended Chandra Deep Field South (ECDFS), a famous extragalactic deep field that was a target in DP1. It is the deepest 6-filter DP1 imaging. Then display using the LSST pipeline package afwDisplay.

In [ ]:
ECDFS_ra = 53.2
ECDFS_dec = -28.1

radius = 0.1
region = sphgeom.Region.from_ivoa_pos(f"CIRCLE {ECDFS_ra} {ECDFS_dec} {radius}")

coadd_datasetrefs = butler.query_datasets("deep_coadd",
                                          where="patch.region OVERLAPS region AND band='r'",
                                          bind={"region": region},
                                          with_dimension_records=True,
                                          order_by=["patch.tract"])

coadd = butler.get(coadd_datasetrefs[0])

In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(coadd.image)
plt.show()


> Figure 1: A `deepCoadd` patch image of the ECDFS field.

The following cell demonstrates how to retrieve the world coordinate system (WCS) information.

In [ ]:
wcs = coadd.getWcs()
print(wcs)

Grab a small cutout of the coadded image at the location of the ECDFS region covering the Hubble Ultra Deep Field (HUDF) to explore further. To do this, use the function `make_image_cutout` which is defined at the beginning of the notebook. This function extracts a cutout from a deep coadd image at a given RA, Dec position and desired image size, using the Rubin image cutout service.

In [ ]:
HUDF_RA = 53.159
HUDF_dec = -27.781

mem = make_image_cutout(HUDF_RA, HUDF_dec, cutout_size=0.05)

cutout_image = ExposureF(mem)
print("The size of the cutout in pixels is: ", cutout_image.image.array.shape)


In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout_image.image)
plt.show()

> Figure 2: A cutout of a `deepCoadd` image, centered at the region of ECDFS that is overlapping the Hubble Ultra Deep Field.

## 2.2 The objectTable

The `objectTable` is the catalog of static objects that are detected with S/N > 5 from the `deepCoadd` images by the LSST Science Pipelines. This section will look at the galaxies in the `objectTable` that was extracted from this specific image. 
The TAP service is the recommended way to retrieve catalog data for a notebook, and there are examples on how to do this in [DP1 tutorials](https://github.com/lsst/tutorial-notebooks/tree/main/DP1).

This section demonstrates how to use TAP to retrieve the catalog data within a polygon defined by the corners of the coadd image.

The following cell extracts the XY values of the corners of the cutout image and converts them to RA, Dec, which are then used as spatial constraints in the query below.

In [ ]:
wcs = cutout_image.getWcs()

x0 = float(cutout_image.getX0())
y0 = float(cutout_image.getY0())
width = cutout_image.getWidth()
height = cutout_image.getHeight()

xcorners = [x0, x0+width, x0+width, x0]
ycorners = [y0, y0, y0+width, y0+width]

ra_corners = []
dec_corners = []

for i in range(len(xcorners)):
    radec = wcs.pixelToSky(xcorners[i], ycorners[i])
    ra_corners.append(radec.getRa().asDegrees())
    dec_corners.append(radec.getDec().asDegrees())

Create a query statement for `Objects` within the bounding box corners.

> **Notice:** You may be familiar with the `BETWEEN ... AND ...` format for ADQL spatial queries, and tempted to use it here to select on RA and Dec ranges. However, the Qserv database does not know how to deal with this construction efficiently, so you are _much_ better off constructing a polygon from the corners of the image as below.

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, x, y, tract, patch " + \
        "FROM dp1.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " +\
        "POLYGON('ICRS', " + str(ra_corners[0]) + ", " + str(dec_corners[0]) + ", " +\
        str(ra_corners[1]) + ", " + str(dec_corners[1]) + ", " +\
        str(ra_corners[2]) + ", " + str(dec_corners[2]) + ", " +\
        str(ra_corners[3]) + ", " + str(dec_corners[3]) + ")) = 1 "


Run the query as an asynchronous job, then fetch the results.

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
print(len(results))

Convert the results to an Astropy table, then take a look at the table.

In [ ]:
tab = results.to_table()

Display the image cutout, and overplot the sources on the image. Use display buffering to avoid re-drawing the image after each source is plotted.

In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout_image.image)
with display.Buffering():
    for i in range(len(tab)):
        display.dot('+', tab[i]['x'], tab[i]['y'], ctype=afwDisplay.RED)

plt.show()


> Figure 3: A cutout of a `deepCoadd` image, centered on the HUDF region of ECDFS, now with the location of detected galaxies from the objectTable overplotted.

# 3. Integrated photometry of galaxies

This section will explore integrated photometry measurements generated by the LSST pipeline, and provide some guidance for which are optimal for certain science applications using galaxies. 


Photometry is the measurement of how much light is apparent from astronomical sources. The amount of light arriving on the telescope from the object is typically referred to as the flux density, or apparent magnitude (depending on units). Flux density is defined as the amount of energy arriving on the telescope per unit area, per unit time, per unit frequency (or wavelength) of the light.

The LSST Science Pipelines makes a variety of photometric measurements for point-like and extended sources. This notebook will teach the user about the automated photometry measurements that are measured on the deepCoadd images and appear in the Object Catalog as part of the LSST pipelines data products. 

Please note that these explanations are specific to DP1: the `objectTable` contents and definitions are still subject to change for DP2, and may be different from DP0.2. The photometry in the catalog are flux densities in units of nano-Jansky [nJy]. 1 Jy = 10^{-23} ergs/s/cm^2/Hz.


## 3.1 Types of photometry

Schema for the object catalog for DP1 <a href="https://sdm-schemas.lsst.io/dp1.html#Object">is available here</a>.

Numerous photometry measurements are produced by the LSST Pipelines. Two types of photometry are there. The first are total fluxes, which aim to approximate (or model) all of the light coming from objects. Measurements that approximate total flux are "Composite Model" (cModel) fluxes. The second class of fluxes are measured inside an on-sky aperture but not corrected for flux that may fall outside: thus they are apparent fluxes but do not recover the intrisic (total) flux (see Section 3.2 below). The apparent fluxes are optimized for other purposes, such as for measuring accurate light profiles or accurate colors.




First, see what is available in the object catalog by querying the `tap_schema` columns, and printing all the parameters available related to "Flux" measured in the i-band (as an example). For clarity, the return also omits errors and flags associated with the photometric measurements outlined in section 1.1. 

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp1.Object'"

results = service.search(query).to_table()


In [ ]:
search_string = 'Flux'
band = 'i_'
exclude1 = 'Err'
exclude2 = 'flag'
for cname in results['column_name']:
    if cname.find(search_string) > -1 and cname.find(band) > -1 and \
       cname.find(exclude1) == -1 and cname.find(exclude2) == -1:

        print(cname)

## 3.2 Compare photometric measurements

### 3.2.1 Total fluxes

These are measurements that model the total light of galaxies. They are intended to be used for e.g. luminosity or mass measurements that require the full light output of the galaxy.

#### Sersic fluxes

This photometric measurement models all galaxies as a single Sersic profile and calculates its total flux according to the best fitting model. More information the sersic profile <a href="https://en.wikipedia.org/wiki/S%C3%A9rsic_profile">is available here</a>.

The best fit sersic model is evaluated based on a model to all filters, and then the flux in each filter is calculated by integrating the galaxy light from the best fit model, assuming the best-fit sersic shape parameters. 

```
<f>_sersicFlux    : Flux from the final sersic fit. Forced on <f>-band.
<f>_sersicFluxErr : Uncertainty of <f>_sersicFlux
sersic_no_data_flag   : Failure flag for <f>_sersicFlux
```

DP1 is the first Rubin data release that contains sersic fluxes. cModel fluxes (see below) have seen more testing and been in use longer since they were included in DP0.2. The LSST pipeline package responsible for sersic fluxes is called `multiprofit` and documentation is <a href="https://pipelines.lsst.io/modules/lsst.multiprofit/index.html#module-lsst.multiprofit">available here</a>.


#### Composite Model (CModel) fluxes

Similar in nature to those measured for SDSS (information <a href="https://www.sdss3.org/dr8/algorithms/magnitudes.php#cmodel">available here</a>). 


In short, it is the linear combination of the best fit exponential (disk or D) and de Vaucouleurs (bulge or B) profiles. 

```
<f>_cModelFlux    : Flux from the final cmodel fit. Forced on <f>-band.
<f>_cModelFluxErr : Uncertainty of <f>_cModelFlux
<f>_cModel_flag   : Failure flag for <f>_cModelFlux
```

For most cases the "fixed" cModel photometry (as listed above) are preferred to that measured with more degrees of freedom labeled `<f>_free_cModelFlux`. The difference is that the fixed ones above uses a reference band (recorded as `refBand` in the schema) where the galaxy is well detected to determine the other parameters, which are then fixed when fitting for the flux in the other bands. The `_free_cModelFlux` measurement allows all parameters to be free and independent in each filter. The fixed `_cModelFlux` measurements are generally recommended for galaxy science applications where total flux measurements are needed (e.g. for intrinsic luminosity or mass).



### 3.2.2 Apparent fluxes

Apparent fluxes refer to measurements that are measured but are not corrected to the total flux. These are not intended to be used for luminosity or mass measurements but can be used for color or shape analysis.

#### Kron fluxes

A decent summary of Kron fluxes <a href="https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2_6.html">in the NED documentation</a>. The aperture used for the fluxes is 2.5 x R1 where R1 is the luminosity weighted radius (also called "first moment"; Kron et al. 1980).

```
<f>_kronFlux      : Flux from Kron Flux algorithm. Measured on <f> g-band.
<f>_kronFluxErr   : Uncertainty of <f>_kronFlux.
<f>_kronFlux_flag : Failure flag for <f>_kronFlux.
```

The Kron radius, `<f>_kronRad`, is also available. In this case of LSST pipeline output, the Kron flux is not corrected for light that is emitted outside of the Kron aperture. While in many cases it will collect the majority of light, it will not be as accurate as the cModel for science cases requiring total flux.


#### Aperture fluxes
This contains the enclosed flux inside a given aperture (they are not corrected to total fluxes using an aperture correction that accounts for the flux falling outside the aperture). Fixed aperture size refers to the aperture radius in pixels.

```
<f>_ap<pix>Flux     : Flux within <pix>-pixel aperture. Forced on <f>-band.
<f>_ap<pix>FluxErr  : Uncertainty of <f>_ap<pix>Flux.
<f>_ap<pix>FluxFlag : Failure flag for <f>_ap<pix>Flux.
```

For DP1, the apertures are 3, 6, 9, 12, 17, 25, 35, 50, and 70 pixels. In the column name, apertures are `03`, `06`, `09`, `12`, and so on. While aperture fluxes are not corrected for the loss outside the aperture, if the aperture size is much larger than the galaxy size then it will approximate the total flux of the galaxy. The general application of these measurements are for measuring radial profiles.




## 3.3 Identify galaxies in ECDFS

Below, query the DP1 `objectTable` for a selection of photometric measurements. This section will focus on 4 measurements: total flux from cModel (sum of bulge and disk sersic compoents fitted to the galaxy); and apparent fluxes measured as Kron (typically includes more than 90% of intrinsic light), GaaP (which is optimized for measuring accurate colors between bands), and aperture photometry (flux measured inside circles of varying size).

First, focus on the center of the ECDFS. Limit the search to contain galaxies using the `i_extendedness` flag (which will exclude point sources). Further, identify objects which have been detected at high signal to noise > 20 and whose photometric measurements have not been flagged as having an issue (`i_kronFlux_flag` or `i_cModel_flag` = 0 means the photometry is ok). 

In [ ]:
query = "SELECT obj.objectId, obj.coord_ra, obj.coord_dec, " + \
        "obj.i_blendedness, obj.i_extendedness, " + \
        "obj.i_kronFlux, obj.i_kronFluxErr, obj.i_kronRad, " + \
        "obj.i_cModelFlux, obj.i_cModelFluxErr, " + \
        "obj.i_gaap1p0Flux, obj.g_gaap1p0Flux, " + \
        "obj.sersic_index, obj.i_sersicFlux, obj.sersic_reff_x, " + \
        "obj.sersic_reff_y, obj.sersic_rho, " + \
        "obj.i_kronFlux_flag, obj.i_cModel_flag, " + \
        "obj.shape_xx, obj.shape_xy, obj.shape_yy, " + \
        "obj.i_ap06Flux, obj.i_ap09Flux, obj.i_ap12Flux, " + \
        "obj.i_ap17Flux, obj.i_ap35Flux " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (obj.i_cModelFlux/obj.i_cModelFluxErr > 20) AND " + \
        "(obj.i_extendedness = 1) AND " + \
        "(obj.i_kronFlux_flag = 0) AND (obj.i_cModel_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS',"+str(ECDFS_ra)+","+str(ECDFS_dec)+", 0.1)) = 1 "

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Print the results of the search query.

In [ ]:
results = job.fetch_result()
tab = results.to_table()
tab

Below, store the Kron Radius, which is a good proxy for the size of the galaxy light profile. Then convert the fluxes extracted from the `objectTable` into AB magnitudes

> **Warning:** The following cell will produce warnings for invalid value encountered in log10, which happens if the source flux is negative. This occasionally happens from aperture photometry if the included pixels inside the aperture have negative values and can be safely ignored for this example.

In [ ]:
i_kronRad = tab['i_kronRad']

cmodel_mag = -2.50 * np.log10(tab['i_cModelFlux']) + 31.4
sersic_mag = -2.50 * np.log10(tab['i_sersicFlux']) + 31.4

ap06_mag = -2.50 * np.log10(tab['i_ap06Flux']) + 31.4
ap09_mag = -2.50 * np.log10(tab['i_ap09Flux']) + 31.4
ap17_mag = -2.50 * np.log10(tab['i_ap17Flux']) + 31.4
ap12_mag = -2.50 * np.log10(tab['i_ap12Flux']) + 31.4

## 3.4 Visualize photometric apertures

Here make a cutout of a large galaxy in the cluster and compare the photometric aperture sizes of the different photometric measurements. The cell below also demonstrates how to reconstruct the Kron aperture using the corresponding shape parameters in the `objectTable`. The position angle `theta` is defined in radians counterclockwise from the x-axis. This cell will plot for an elongated edge-on galaxy (`objectId` of 611254385447537498) to ensure that the visualization emphasizes the difference between the elliptical Kron aperture to the circular aperture. 

Note that the pixel scale of the LSST data is 0.2 arcseconds per pixel.


In [ ]:
wh = np.where((tab['objectId'] == 611254385447537498))[0]
indx = 0

arcsec_per_pix = 0.2

In the next cell, use `objectTable` shape parameters to reconstruct the Kron shape. The method of the Kron implementation in the LSST pipelines is parameterized by three shape parameters `shape_xx`, `shape_yy`, and `shape_xy`, which must be converted using LSST package `ellipses`. The cell below will demonstrate how to extract the more commonly used semi-major and semi-minor axes (`Rmaj`, `Rmin`) and the rotation angle `theta`. The Kron Radius `kronRad` is defined as sqrt(Rmaj * Rmin). 

In [ ]:
axes = ellipses.Axes(ellipses.Quadrupole(tab['shape_xx'][wh][indx],
                                         tab['shape_yy'][wh][indx],
                                         tab['shape_xy'][wh][indx]))
Rmaj = axes.getA()
Rmin = axes.getB()
theta = axes.getTheta()

Generate an image cutout.

In [ ]:
mem = make_image_cutout(tab['coord_ra'][wh][indx],
                        tab['coord_dec'][wh][indx], cutout_size=0.005)

cutout = ExposureF(mem)

Plot the cutout and overplot the various shapes and apertures.

In [ ]:
plt.subplot(projection=WCS(cutout.getWcs().getFitsMetadata()))
extent = (cutout.getBBox().beginX, cutout.getBBox().endX,
          cutout.getBBox().beginY, cutout.getBBox().endY)

plt.imshow(cutout.image.array, extent=extent,
           origin='lower', cmap='gray')

coord = SkyCoord(ra=tab['coord_ra'][wh][indx]*u.degree,
                 dec=tab['coord_dec'][wh][indx]*u.degree, frame='icrs')

aperture = SkyCircularAperture(coord, r=17 * arcsec_per_pix * u.arcsec)
pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='r', lw=3, label='17 pixel aperture')

aperture = SkyCircularAperture(coord, r=35 * arcsec_per_pix * u.arcsec)
pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='g', lw=3, label='35 pixel aperture')

aperture = SkyEllipticalAperture(coord,
                                 2.5 * Rmaj * arcsec_per_pix * u.arcsec,
                                 2.5 * Rmin * arcsec_per_pix * u.arcsec,
                                 theta=(np.pi/2 + theta) * u.rad)

pix_aperture = aperture.to_pixel(WCS(cutout.getWcs().getFitsMetadata()))
pix_aperture.plot(color='b', lw=3, label='Kron')
plt.legend()

> Figure 4: An i-band image cutout of an example galaxy (grayscale). The photometric apertures are overplotted including Kron (blue), the 17 pixel aperture (red) and the 35 pixel aperture (green). None of the apertures enclose all of the flux visible from the galaxy, which is why the photometry measures less flux than the total flux from cModel.

The above figure shows more clearly that there is some fraction of the galaxy's light excluded by each of these photometric apertures. Further, printing the different photometry measurements illustrates how these assumed apertures will change the measured flux (compared to the total flux measured by cModel, which approximates the total magnitude using sersic models for the light profile).  

## 3.5 Build a sersic model

The sersic model used to measure `sersicFlux` in the `objectTable` can be reconstructed using the sersic shape parameters. These include the `sersic_reff_x` and `sersic_reff_y` which are the effective radii (along the x- and y-axis) from the multiband Sersic model fit, prior to convolution with the point spread function (PSF); `sersic_rho` which is the correlation coefficient from the multiband Sersic model fit. These can be converted to the more commonly used axis ratio `ba` (ratio of semi-major and semi-minor axes), the position angle `pa` which indicates the rotation counter clockwise with respect to the x-axis. 


In [ ]:
x = EllipseMajor(Ellipse(sigma_x=tab['sersic_reff_x'][wh][indx],
                         sigma_y=tab['sersic_reff_y'][wh][indx],
                         rho=tab['sersic_rho'][wh][indx]), degrees=True)
ba = x.axrat
pa = x.angle
r_major = x.r_major * arcsec_per_pix


The next cell will use `galsim` `Sersic` function to create a 2D sersic model of the galaxy. Finally, the `sersic_index` which indicates the shape of the light profile is used in addition to the ellipse parameters retrieved in the cell above.

In [ ]:
sersic = gs.Sersic(n=tab['sersic_index'][wh][indx], half_light_radius=r_major*np.sqrt(ba),
                   flux=tab['i_sersicFlux'][wh][indx]).shear(q=ba, beta=pa*gs.degrees)

img = sersic.drawImage(nx=50, ny=50, method="real_space", scale=arcsec_per_pix).array

print('Total flux of model = ', np.sum(img),
      ' which equals flux from objectTable = ',
      tab['i_sersicFlux'][wh][indx])

fig, ax = plt.subplots()
im = ax.imshow(img, origin='lower', interpolation='nearest', vmin=0.01, vmax=101)

plt.xlabel('x')
plt.ylabel('y')
plt.show()

> Figure 5: The best fit Sersic model of the galaxy in Figure 4 that was used to measure `sersicFlux` of the galaxy. 

## 3.6 Compare aperture to total fluxes

This section will make several plots that shows how the different photometric measurements compare. Note that cModel is the total flux integrated from a model fit to the galaxy light profile, while aperture photometry are measures of light within a fixed aperture.  

### 3.6.1 Aperture photometry

Generally, magnitudes measured using aperture photometry in the LSST pipeline are larger (i.e. fainter) than those measured from cModel, becuase the fixed circular aperture systematically underestimates the flux in the galaxy wings (and the lost flux increases as the intrinsic size of the galaxy increases, e.g. as traced by the Kron radius).

In [ ]:
fig, (ax, ax2) = plt.subplots(ncols=2, nrows=1,
                              width_ratios=[0.8, 0.2], figsize=(10, 6))

ylims = [-1.5, 1.5]
ax.plot(i_kronRad, (cmodel_mag-ap06_mag), '^', alpha=.3,
        label='6-pixel aperture', color='red')
ax.plot(i_kronRad, (cmodel_mag-ap09_mag), 's', alpha=.3,
        label='9-pixel aperture', color='orange')
ax.plot(i_kronRad, (cmodel_mag-ap12_mag), 'o', alpha=.3,
        label='12-pixel aperture', color='green')
ax.plot(i_kronRad, (cmodel_mag-ap17_mag), '.', alpha=.3,
        label='17-pixel aperture', color='blue')

ax2.hist((cmodel_mag-ap17_mag), edgecolor='blue', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap12_mag), edgecolor='green', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap09_mag), edgecolor='orange', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)
ax2.hist((cmodel_mag-ap06_mag), edgecolor='red', orientation="horizontal",
         bins=np.linspace(ylims[0], ylims[1], 40), align='mid',
         histtype="step", stacked=True, fill=False)

ax2.set_ylim(ylims)
ax.axhline(0, linestyle='--')
ax.set_xlabel('Kron Radius [i-band; pixels]')
ax.set_ylabel('cModel mag - Aperture mag')
ax.set_ylim(ylims)
ax.set_xlim([2, 10])
ax.legend()


> Figure 6: A comparison of the difference between cModel photometry and aperture photometry measured by the LSST pipelines for four different aperture sizes as a function of galaxy size (as measured using the Kron radius). The left panel shows the scatter plot of difference in photometry vs Kron radius, and the right panel shows a histogram of these values that demonstrate that larger aperture sizes have photometry that is closer to the cModel.

This figure shows that the aperture photometry typically under-estimates the flux relative to the total flux estimated using cModel. As expected, there is a general trend that using larger apertures gets closer to the total flux from cModel, for larger galaxies (i.e. whose Kron Radius is larger). 


# 4. Science application: galaxy colors

This section demonstrates using GaaP photometry to calculate accurate galaxy colors to identify different types of galaxies. 

#### GaaP fluxes

These are optimized for measuring accurate colors between bands (the Gaussian-aperture-and-PSF flux, defined in <a href="https://ui.adsabs.harvard.edu/abs/2008A%26A...482.1053K/abstract">Kuijken et al. 2008</a>). The main goal of this method is to measure accurate colors while accounting for the different spatial resolution between filters. This is sometimes achieved in other datasets by convolving all images to the largest PSF, but this process of PSF-matching is computationally very time consuming for large images, thus motivating GaaP as a faster alternative. It is not a measure of total flux in a filter. Several measurement apertures are available. 

**Aperture**

```
<f>_gaap<ap>Flux    : GaaP flux with <ap> aperture after multiplying the seeing aperture. Forced on <f>-band.
<f>_gaap<ap>FluxErr : Uncertainty of <f>_gaap<ap>Flux.
```

Where the measurement apertures are 0.5, 0.7, 1.0, 1.5, 2.5, and 3.0 arcseconds. In the column name `<ap>` appears as `0p5`, `0p7`, etc. Multiplying by the "seeing aperture" refers to convolving the PSF with a kernel so that the PSF is as if the seeing were 1.15 arcseconds. This has the effect of smearing the images of all filters consistently so that the colors are accurate.

For photometric redshifts, and other analysis where accurate colors are important, it is recommended to start with the GaaP fluxes with 1.0 aperture (optimal aperture was found to not perform as well, and should not be used). The largest aperture `gaap3p0` might work better for larger galaxies, but `gaap1p0` has better overall performance. Experiment yourself to see how it works for your science case.




## 4.1 Identify cluster galaxies

In galaxy clusters, galaxies tend to be old, red elliptical galaxies and thus exhibit a well defined red sequence in color space, while field environments (e.g. the ECDFS region), the galaxy population is typically dominated by bluer star forming galaxies. 

In the cell below, query for a second sample of galaxies that live in a galaxy cluster. The Abell 360 cluster sits inside one of the DP1 footprints, so we will search for galaxies around that cluster.


In [ ]:
cluster_RA = 37.83
cluster_dec = 6.98

query = "SELECT obj.objectId, obj.coord_ra, obj.coord_dec, " + \
        "obj.i_blendedness, obj.i_extendedness, " + \
        "obj.i_cModelFlux, obj.i_cModelFluxErr, " + \
        "obj.g_gaap1p0Flux, obj.i_gaap1p0Flux, " + \
        "obj.i_kronFlux_flag, obj.i_cModel_flag " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (obj.i_cModelFlux/obj.i_cModelFluxErr > 20) AND " + \
        "(obj.i_extendedness = 1) AND " + \
        "(obj.i_kronFlux_flag = 0) AND (obj.i_cModel_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS',"+str(cluster_RA)+","+str(cluster_dec)+", 0.1)) = 1 "


In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)


In [ ]:
results = job.fetch_result()
tab2 = results.to_table()



First, calculate the g, r, i magnitudes of galaxies in the galaxy cluster, and also for other filters not used yet (g, r) from a 'field' environment (use the ECDFS galaxies from the query performed in Section 2). This will enable plotting a comparison of their colors, which is performed in the second cell below.

> **Warning:** Like in Section 2.3, the following cell will produce warnings for invalid value encountered in log10, which happens if the source flux is negative. This happens for a small number of objects and since the goal of the plot is to see the distribution of the majority of sources, the warning can be safely ignored. 

In [ ]:
i_cluster_gaap_mag = -2.50 * np.log10(tab2['i_gaap1p0Flux']) + 31.4
g_cluster_gaap_mag = -2.50 * np.log10(tab2['g_gaap1p0Flux']) + 31.4

i_ECDFS_gaap_mag = -2.50 * np.log10(tab['i_gaap1p0Flux']) + 31.4
g_ECDFS_gaap_mag = -2.50 * np.log10(tab['g_gaap1p0Flux']) + 31.4


In [ ]:
fig, (ax, ax1) = plt.subplots(ncols=1, nrows=2, figsize=(10, 6))

ax.plot(i_cluster_gaap_mag, (g_cluster_gaap_mag-i_cluster_gaap_mag),
        '.', alpha=.1, color='r', label='Cluster A360 Galaxies')
ax.set_xlabel('i-band Magnitude [GaaP]')
ax.set_ylabel('g-i color')
ax.set_ylim([-1, 4])
ax.legend()

ax1.plot(i_ECDFS_gaap_mag, (g_ECDFS_gaap_mag-i_ECDFS_gaap_mag),
         '.', alpha=.1, color='b', label='Field ECDFS Galaxies')
ax1.set_xlabel('i-band Magnitude [GaaP]')
ax1.set_ylabel('g-i color')
ax1.set_ylim([-1, 4])
ax1.legend()

> Figure 7: Color magnitude diagram constructed using GaaP fluxes for a cluster (top panel) and a field population (bottom panel). A very nice red sequence appears from the red, old galaxies in the cluster (top panel) while field galaxies (bottom panel) are dominated by bluer objects and do not exhibit a red sequence.  